In [108]:
import numpy as np
import pandas as pd
import tensorflow as tf

rawData = pd.read_pickle("daily_temp.pkl").dropna()

print(rawData.columns)
data = rawData[rawData['Total Snow'].notna()].values

trainingPortion = 0.8

np.random.shuffle(data)

splitIndex = int(len(data) * trainingPortion)

print(data)



Index(['Longitude', 'Latitude', 'Year', 'Month', 'Day', 'Max Temp', 'Min Temp',
       'Mean Temp', 'Heat Deg Days', 'Cool Deg Days', 'Total Rain',
       'Total Snow', 'Total Precip'],
      dtype='object')
[[-1.2773e+02  5.0980e+01  2.0060e+03 ...  1.4000e+00  0.0000e+00
   1.4000e+00]
 [-1.2513e+02  4.9880e+01  1.9740e+03 ...  0.0000e+00  0.0000e+00
   0.0000e+00]
 [-1.1735e+02  4.9480e+01  1.9210e+03 ...  0.0000e+00  0.0000e+00
   0.0000e+00]
 ...
 [-1.2662e+02  5.5320e+01  1.9740e+03 ...  0.0000e+00  0.0000e+00
   0.0000e+00]
 [-1.2020e+02  5.1260e+01  1.9990e+03 ...  0.0000e+00  0.0000e+00
   0.0000e+00]
 [-1.2413e+02  4.8820e+01  1.9810e+03 ...  0.0000e+00  0.0000e+00
   0.0000e+00]]
[[ 7.   5.   1.4  0. ]
 [20.  11.7  0.   0. ]
 [24.4  0.6  0.   0. ]
 ...
 [10.  -7.2  0.   0. ]
 [ 3.9  0.6  0.   0. ]
 [10.6  1.7  0.   0. ]]


In [123]:
outputCols = [4,6,10]
normalizeValue = data[:, 4:7].max(axis=0)
#minValue = data[]

X_train = data[:splitIndex,0:5]
Y_train = data[:splitIndex, outputCols]
#Y_train = Y_train/normalizeValue

X_test = data[splitIndex:, 0:5]
Y_test = data[splitIndex:, outputCols]
#Y_test = Y_test/normalizeValue

print(X_train)

[[-127.73   50.98 2006.     12.     30.  ]
 [-125.13   49.88 1974.      6.     21.  ]
 [-117.35   49.48 1921.      9.     17.  ]
 ...
 [-122.7    49.15 1954.      3.     29.  ]
 [-117.63   49.3  2001.      3.      5.  ]
 [-121.72   52.26 1976.     10.     11.  ]]


In [107]:
x = [10,20,30,40,50]
x[3 and 4]


50

In [120]:
print(X_train[:3])
print(Y_test[:3])
print(normalizeValue)
#data[:, 6].argmax()
#data[188,:]
#print(X_train.shape)

[]
[[27.   5.   3.8]
 [20.  11.   0. ]
 [23.  10.   0. ]]
[31.  50.  31.7]


In [135]:
inputShape = [X_train.shape[1]]

model = tf.keras.Sequential([
    tf.keras.Input(shape=inputShape),
    tf.keras.layers.Dense(units=1024, activation="softsign"),
    tf.keras.layers.Dense(units=512, activation="relu"),
    tf.keras.layers.Dense(units=256, activation="relu"),
    #tf.keras.layers.Dense(units=128, activation="tanh"),
    tf.keras.layers.Dense(units=3, activation="linear")
])

#model.compile(optimizer='adam', loss="mean_squared_logarithmic_error", metrics=['mean_squared_logarithmic_error'])
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredLogarithmicError(), 'accuracy'])
fitModel = model.fit(X_train, Y_train, epochs=20, validation_data=[X_test, Y_test], batch_size=256)

Epoch 1/20
25942/25942 [==============================] - 129s 5ms/step - loss: 27.9256 - mean_squared_logarithmic_error: 0.5875 - accuracy: 0.8749 - val_loss: 28.2384 - val_mean_squared_logarithmic_error: 0.6919 - val_accuracy: 0.8701
Epoch 2/20
25942/25942 [==============================] - 125s 5ms/step - loss: 26.2661 - mean_squared_logarithmic_error: 0.5425 - accuracy: 0.8806 - val_loss: 26.2950 - val_mean_squared_logarithmic_error: 0.5177 - val_accuracy: 0.8791
Epoch 3/20
25942/25942 [==============================] - 130s 5ms/step - loss: 26.0975 - mean_squared_logarithmic_error: 0.5373 - accuracy: 0.8809 - val_loss: 26.1223 - val_mean_squared_logarithmic_error: 0.5350 - val_accuracy: 0.8760
Epoch 4/20
25942/25942 [==============================] - 130s 5ms/step - loss: 25.9813 - mean_squared_logarithmic_error: 0.5354 - accuracy: 0.8811 - val_loss: 25.7999 - val_mean_squared_logarithmic_error: 0.5226 - val_accuracy: 0.8797
Epoch 5/20
25942/25942 [==============================] 

In [148]:
print(inputShape)
print(model.predict([[-123, 45.5, 4695, 10, 30]]))

[5]
1/1 [==============================] - 0s 49ms/step
[[14.319339  14.863642   0.4450842]]


In [109]:
decentTempModel = model
decentTempModelHist = fitModel

In [ ]:
inputShape = [X_train.shape[1]]

model = tf.keras.Sequential([
    tf.keras.Input(shape=inputShape),
    tf.keras.layers.Dense(units=1024, activation="relu"),
    tf.keras.layers.Dense(units=512, activation="relu"),
    tf.keras.layers.Dense(units=256, activation="relu"),
    tf.keras.layers.Dense(units=128, activation="relu"),
    tf.keras.layers.Dense(units=7, activation="relu")
])

#model.compile(optimizer='adam', loss="mean_squared_logarithmic_error", metrics=['mean_squared_logarithmic_error'])
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredLogarithmicError(), metrics=[tf.keras.metrics.MeanAbsolutePercentageError(), 'accuracy'])
model.fit(X_train, Y_train, epochs=3, validation_data=[X_test, Y_test], batch_size=1024)

Epoch 1/3
6486/6486 [==============================] - 35s 5ms/step - loss: 0.0284 - mean_absolute_percentage_error: 1327996.5000 - accuracy: 0.3100 - val_loss: 0.0293 - val_mean_absolute_percentage_error: 62.5130 - val_accuracy: 0.3919
Epoch 2/3
6486/6486 [==============================] - 33s 5ms/step - loss: 0.0274 - mean_absolute_percentage_error: 62.5063 - accuracy: 0.3100 - val_loss: 0.0293 - val_mean_absolute_percentage_error: 62.5130 - val_accuracy: 0.3919
Epoch 3/3
6486/6486 [==============================] - 31s 5ms/step - loss: 0.0274 - mean_absolute_percentage_error: 62.5063 - accuracy: 0.3100 - val_loss: 0.0293 - val_mean_absolute_percentage_error: 62.5130 - val_accuracy: 0.3919
